# ***6530611033 Project ML ข้อที่ 1***


In [ ]:
# Access to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# library

In [ ]:
import librosa
import numpy as np
import os
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from concurrent.futures import ProcessPoolExecutor

โหลดข้อมูล

In [ ]:
glass1_path = '/content/drive/MyDrive/Colab Notebooks/project_baselearning/แก้ว 1'
glass2_path = '/content/drive/MyDrive/Colab Notebooks/project_baselearning/แก้ว 2'

เปลี่ยนไฟล์ wav to mp3

In [ ]:
!pip install pydub
from pydub import AudioSegment

def convert_wav_to_mp3(wav_path, mp3_path):
  try:
    sound = AudioSegment.from_wav(wav_path)
    sound.export(mp3_path, format="mp3")
    os.remove(wav_path)  # Delete the WAV file
    print(f"Converted {wav_path} to {mp3_path} and deleted the WAV file.")
  except Exception as e:
    print(f"An error occurred: {e}")

directory = "/content/drive/MyDrive/Colab Notebooks/project_baselearning/แก้ว 1/"

for filename in os.listdir(directory):
  if filename.endswith(".wav"):
    wav_file_path = os.path.join(directory, filename)
    mp3_file_path = os.path.join(directory, filename[:-4] + ".mp3")
    convert_wav_to_mp3(wav_file_path, mp3_file_path)

directory = "/content/drive/MyDrive/Colab Notebooks/project_baselearning/แก้ว 2/"
for filename in os.listdir(directory):
  if filename.endswith(".wav"):
    wav_file_path = os.path.join(directory, filename)
    mp3_file_path = os.path.join(directory, filename[:-4] + ".mp3")
    convert_wav_to_mp3(wav_file_path, mp3_file_path)

# **โหลดข้อมูลสำหรับทำ feature vector**

MFCC สร้าง feature_vector

In [ ]:
# ฟังก์ชันดึงคุณสมบัติจากไฟล์เสียง
def extract_features(file_path):
    try:
        # โหลดไฟล์เสียง
        y, sr = librosa.load(file_path, duration=3.0)
        # ดึงคุณสมบัติต่าง ๆ
        mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T, axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0)
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr).T, axis=0)
        tonnetz = np.mean(librosa.feature.tonnetz(y=y, sr=sr).T, axis=0)
        zero_crossings = np.mean(librosa.feature.zero_crossing_rate(y=y).T, axis=0)
        rms = np.mean(librosa.feature.rms(y=y).T, axis=0)
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr).T, axis=0)
        mel = np.mean(librosa.feature.melspectrogram(y=y, sr=sr).T, axis=0)
        return np.hstack([mfcc, chroma, spectral_contrast, tonnetz,
                          zero_crossings, rms, spectral_rolloff, mel])
    except:
        return None

In [ ]:
features = []
labels = []

# ดึง features สำหรับแก้ว 1
for filename in tqdm(os.listdir(glass1_path)):
    if filename.endswith(".mp3"):
        file_path = os.path.join(glass1_path, filename)
        features.append(extract_features(file_path))
        labels.append(0)

# ดึง features สำหรับแก้ว 2
for filename in tqdm(os.listdir(glass2_path)):
    if filename.endswith(".mp3"):
        file_path = os.path.join(glass2_path, filename)
        features.append(extract_features(file_path))
        labels.append(1)

# Convert lists to NumPy arrays
X = np.array(features)
y = np.array(labels)


100%|██████████| 51/51 [00:14<00:00,  3.49it/s]


In [ ]:
# ตรวจสอบว่ามีข้อมูลหรือไม่
if len(features) > 0:
    # แปลงข้อมูลเป็นอาเรย์และมาตรฐานข้อมูล
    X = np.array(features)
    y = np.array(labels)

    # แบ่งข้อมูลเป็น Training และ Testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # ทำการมาตรฐานข้อมูล
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # การใช้ SMOTE เพื่อทำ oversampling ข้อมูลที่ไม่สมดุล
    smote = SMOTE(random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

    # ปรับปรุงพารามิเตอร์ของ Decision Tree และเพิ่มการตั้งค่าใน Random Forest เพื่อให้ได้ประสิทธิภาพที่ดีขึ้น
    classifiers = {
        "Decision Tree": DecisionTreeClassifier(max_depth=150, min_samples_split=2, min_samples_leaf=1, random_state=42),
        "Random Forest": RandomForestClassifier(n_estimators=1500, max_depth=100, min_samples_split=3, random_state=42),
        "Naive Bayes": GaussianNB(),
        "Neural Network": MLPClassifier(hidden_layer_sizes=(200, 100), max_iter=2000, alpha=0.01, random_state=42),
        "Logistic Regression": LogisticRegression(C=0.1, max_iter=500, solver='liblinear', random_state=42)
    }


In [ ]:
results = {}
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    print(f"{name}: Accuracy = {accuracy}")
    print(classification_report(y_test, y_pred))

Decision Tree: Accuracy = 0.6190476190476191
              precision    recall  f1-score   support

           0       0.86      0.46      0.60        13
           1       0.50      0.88      0.64         8

    accuracy                           0.62        21
   macro avg       0.68      0.67      0.62        21
weighted avg       0.72      0.62      0.61        21

Random Forest: Accuracy = 0.7619047619047619
              precision    recall  f1-score   support

           0       0.90      0.69      0.78        13
           1       0.64      0.88      0.74         8

    accuracy                           0.76        21
   macro avg       0.77      0.78      0.76        21
weighted avg       0.80      0.76      0.77        21

Naive Bayes: Accuracy = 0.6190476190476191
              precision    recall  f1-score   support

           0       0.65      0.85      0.73        13
           1       0.50      0.25      0.33         8

    accuracy                           0.62      